In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
uid_train = pd.read_csv('../data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../data/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../data/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../data/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [4]:
prefix = np.array(['u'])
uid_num = np.arange(7000,10000)
uid_num_char = uid_num.astype('U')
uid_num_str = np.core.defchararray.add(prefix, uid_num_char)
uid_test = pd.DataFrame(uid_num_str, columns=['uid'])
uid_test.to_csv('../data/uid_test_a.txt',index=None)

In [5]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

In [6]:
voice.start_time = voice.start_time.astype(int)
voice.end_time = voice.end_time.astype(int)
voice['date'] = voice.start_time//1000000
voice['hour'] = voice.start_time%1000000//10000

sms.start_time = sms.start_time.astype(int)
sms['date'] = sms.start_time//1000000
sms['hour'] = sms.start_time%1000000//10000

wa.date = wa.date.fillna(0).astype(int)
wa.up_flow = wa.up_flow.fillna(0).astype(int)
wa.down_flow = wa.down_flow.fillna(0).astype(int)
wa.visit_dura = wa.visit_dura.fillna(0).astype(int)
wa.visit_cnt = wa.visit_cnt.fillna(0).astype(int)

### 通话记录

In [7]:
voice['voice_dura']=abs(voice.end_time.astype('int')-voice.start_time.astype('int'))
# 不同的电话号码数/电话总数
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()

# 不同的电话号码头三位的数量
voice_opp_head=voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

# 每种电话长度的通话次数
voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

# 每种类型通话的次数
voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

# 每种类型通话的平均时长
voice_dura_type = voice.groupby(['uid','call_type'])['voice_dura'].mean().unstack().add_prefix('voice_dura_type_').reset_index().fillna(0)

# 接入/打出的电话总数
voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)

# 通话时长的各统计量
voice_dura = voice.groupby(['uid'])['voice_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_dura_').reset_index().fillna(0)

## 每个用户收/发电话的号码的不同号码数
voice_opp_len_inout = voice.groupby(['uid','in_out'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).unstack().add_prefix('voice_opp_inout_num_').reset_index().fillna(0)

# 不同的日期数
voice_day_count = voice.groupby(['uid'])['date'].agg({'voice_day_count': lambda x: len(pd.unique(x))}).reset_index().fillna(0)

# 每天in/out电话量
voice_everyday_in_count = voice[voice.in_out==1].groupby(['uid','date'])['uid'].count().unstack().add_prefix('voice_everyday_in_count').reset_index().fillna(0)
voice_everyday_out_count = voice[voice.in_out==0].groupby(['uid','date'])['uid'].count().unstack().add_prefix('voice_everyday_out_count').reset_index().fillna(0)

voice_everyday_in_dura = voice[voice.in_out==1].groupby(['uid','date'])['voice_dura'].sum().unstack().add_prefix('voice_everyday_in_dura').reset_index().fillna(0)
voice_everyday_out_dura = voice[voice.in_out==0].groupby(['uid','date'])['voice_dura'].sum().unstack().add_prefix('voice_everyday_out_dura').reset_index().fillna(0)

# 每个小时段的平均电话量
voice_hour_count1 = voice[voice.in_out==1].groupby(['uid','hour'])['uid'].count().unstack().add_prefix('voice_hour_count_in').reset_index().fillna(0)
voice_hour_count0 = voice[voice.in_out==0].groupby(['uid','hour'])['uid'].count().unstack().add_prefix('voice_hour_count_out').reset_index().fillna(0)
voice_hour_count = pd.merge(voice_hour_count1,voice_hour_count0,how='left',on='uid')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [73]:
# 每天每种类型的电话数量
voice_everyday_call_type_out = voice[voice.in_out==0].groupby(['uid','date','call_type'])['uid'].count().unstack().add_prefix('voice_everyday_call_type_').reset_index().fillna(0)
voice_everyday_call_type_1 = voice_everyday_call_type_out.groupby(['uid','date'])['voice_everyday_call_type_1'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
voice_everyday_call_type_2 = voice_everyday_call_type_out.groupby(['uid','date'])['voice_everyday_call_type_2'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
voice_everyday_call_type_3 = voice_everyday_call_type_out.groupby(['uid','date'])['voice_everyday_call_type_3'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
voice_everyday_call_type_4 = voice_everyday_call_type_out.groupby(['uid','date'])['voice_everyday_call_type_4'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
voice_everyday_call_type_5 = voice_everyday_call_type_out.groupby(['uid','date'])['voice_everyday_call_type_5'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
voice_everyday_call_type_out = pd.merge(voice_everyday_call_type_1,voice_everyday_call_type_2,how='left',on='uid')
voice_everyday_call_type_out = pd.merge(voice_everyday_call_type_out,voice_everyday_call_type_3,how='left',on='uid')
voice_everyday_call_type_out = pd.merge(voice_everyday_call_type_out,voice_everyday_call_type_4,how='left',on='uid')
voice_everyday_call_type_out = pd.merge(voice_everyday_call_type_out,voice_everyday_call_type_5,how='left',on='uid')

voice_everyday_call_type_in = voice[voice.in_out==1].groupby(['uid','date','call_type'])['uid'].count().unstack().add_prefix('voice_everyday_call_type_').reset_index().fillna(0)
voice_everyday_call_type_1 = voice_everyday_call_type_in.groupby(['uid','date'])['voice_everyday_call_type_1'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
voice_everyday_call_type_2 = voice_everyday_call_type_in.groupby(['uid','date'])['voice_everyday_call_type_2'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
voice_everyday_call_type_3 = voice_everyday_call_type_in.groupby(['uid','date'])['voice_everyday_call_type_3'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
#voice_everyday_call_type_4 = voice_everyday_call_type_in.groupby(['uid','date'])['voice_everyday_call_type_4'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
voice_everyday_call_type_5 = voice_everyday_call_type_in.groupby(['uid','date'])['voice_everyday_call_type_5'].sum().unstack().add_prefix('day_').reset_index().fillna(0)
voice_everyday_call_type_in = pd.merge(voice_everyday_call_type_1,voice_everyday_call_type_2,how='left',on='uid')
voice_everyday_call_type_in = pd.merge(voice_everyday_call_type_in,voice_everyday_call_type_3,how='left',on='uid')
#voice_everyday_call_type_in = pd.merge(voice_everyday_call_type_in,voice_everyday_call_type_4,how='left',on='uid')
voice_everyday_call_type_in = pd.merge(voice_everyday_call_type_in,voice_everyday_call_type_5,how='left',on='uid')

voice_everyday_call_type = pd.merge(voice_everyday_call_type_out,voice_everyday_call_type_in,how='left',on='uid')


In [9]:
# 每天in/out不同的电话号码数/电话总数
voice_everyday_opp_num1 = voice[voice.in_out==1].groupby(['uid','date'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).unstack().add_prefix('voice_in_opp_num_day').reset_index().fillna(0)
voice_everyday_opp_num0 = voice[voice.in_out==0].groupby(['uid','date'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).unstack().add_prefix('voice_out_opp_num_day').reset_index().fillna(0)
voice_everyday_opp_num = pd.merge(voice_everyday_opp_num1,voice_everyday_opp_num0,how='left',on='uid')

# 每天in/out不同的头三位数
#voice_everyday_opp_head1 = voice[voice.in_out==1].groupby(['uid','date'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).unstack().add_prefix('voice_everyday_in_opp_head').reset_index().fillna(0)
#voice_everyday_opp_head0 = voice[voice.in_out==0].groupby(['uid','date'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).unstack().add_prefix('voice_everyday_out_opp_head').reset_index().fillna(0)
#voice_everyday_opp_head = pd.merge(voice_everyday_opp_num1,voice_everyday_opp_num0,how='left',on='uid')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


## 短信记录

In [10]:
# 不同的in/out短信号码数/电话总数
sms_out_opp_num = sms[sms.in_out==0].groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_out_opp_num').reset_index()
sms_in_opp_num = sms[sms.in_out==1].groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_in_opp_num').reset_index()

# in/out号码不同头三位的数量
sms_opp_head=sms.groupby(['uid','in_out'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).unstack().add_prefix('sms_opp_head_').reset_index()

# 每种号码长度的短信次数
sms_out_opp_len=sms[sms.in_out==0].groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_out_opp_len').reset_index().fillna(0)
sms_in_opp_len=sms[sms.in_out==1].groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_in_opp_len').reset_index().fillna(0)

# 接受/发出短信总数
sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)

# 不同的日期数
sms_day_count = sms.groupby(['uid'])['start_time'].agg({'sms_day_count': lambda x: len(pd.unique(x//1000000))}).reset_index().fillna(0)

# 每天in/out短信量
sms_everyday_out_count = sms[sms.in_out==0].groupby(['uid','date'])['uid'].count().unstack().add_prefix('sms_everyday_out_count').reset_index().fillna(0)
sms_everyday_in_count = sms[sms.in_out==1].groupby(['uid','date'])['uid'].count().unstack().add_prefix('sms_everyday_in_count').reset_index().fillna(0)

# 每个小时段的平均in/out短信量
sms_hour_out_count = sms[sms.in_out==0].groupby(['uid','hour'])['uid'].count().unstack().add_prefix('sms_hour_out_count').reset_index().fillna(0)
sms_hour_in_count = sms[sms.in_out==1].groupby(['uid','hour'])['uid'].count().unstack().add_prefix('sms_hour_in_count').reset_index().fillna(0)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  app.launch_new_instance()


### 网站/APP记录

In [26]:
# 不同wa数量/wa总数
w_a_name_cnt = wa.groupby(['uid','wa_type'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('w_name_').unstack().reset_index()

# 访问wa次数的各统计量
w_a_visit_cnt = wa.groupby(['uid','wa_type'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('w_a_visit_cnt').unstack().reset_index().fillna(0)

# 访问w/a时长的各统计量
w_a_visit_dura = wa.groupby(['uid','wa_type'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('w_a_visit_dura').unstack().reset_index().fillna(0)

## 每个用户w/a的上/下的流量
w_a_upflow = wa.groupby(['uid','wa_type'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('w_a_upflow_').unstack().reset_index().fillna(0)
w_a_downflow = wa.groupby(['uid','wa_type'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('w_a_downflow_').unstack().reset_index().fillna(0)

# 不同的日期数
wa_day_count = wa.groupby(['uid'])['date'].agg({'wa_day_count': lambda x: len(pd.unique(x))}).reset_index().fillna(0)

# 每天web上传流量
wa_everyday_web_up_flow = wa[wa.wa_type==0].groupby(['uid','date'])['up_flow'].sum().unstack().add_prefix('wa_everyday_web_up_flow').reset_index().fillna(0)

# 每天app上传流量
wa_everyday_app_up_flow = wa[wa.wa_type==1].groupby(['uid','date'])['up_flow'].sum().unstack().add_prefix('wa_everyday_app_up_flow').reset_index().fillna(0)

# 每天web下载流量
wa_everyday_web_down_flow = wa[wa.wa_type==0].groupby(['uid','date'])['down_flow'].sum().unstack().add_prefix('wa_everyday_web_down_flow').reset_index().fillna(0)

# 每天app下载流量
wa_everyday_app_down_flow = wa[wa.wa_type==1].groupby(['uid','date'])['down_flow'].sum().unstack().add_prefix('wa_everyday_app_down_flow').reset_index().fillna(0)

# 每天web访问时长
wa_everyday_web_visit_dura = wa[wa.wa_type==0].groupby(['uid','date'])['visit_dura'].sum().unstack().add_prefix('wa_everyday_web_visit_dura').reset_index().fillna(0)

# 每天app访问时长
wa_everyday_app_visit_dura = wa[wa.wa_type==1].groupby(['uid','date'])['visit_dura'].sum().unstack().add_prefix('wa_everyday_app_visit_dura').reset_index().fillna(0)

# 每天web访问时长
wa_everyday_web_visit_cnt = wa[wa.wa_type==0].groupby(['uid','date'])['visit_cnt'].sum().unstack().add_prefix('wa_everyday_web_visit_cnt').reset_index().fillna(0)

# 每天app访问时长
wa_everyday_app_visit_cnt = wa[wa.wa_type==1].groupby(['uid','date'])['visit_cnt'].sum().unstack().add_prefix('wa_everyday_app_visit_cnt').reset_index().fillna(0)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


In [27]:
# 上传流量最多的w名字
wa_w = wa[wa.wa_type==0]
wa_name = wa_w.groupby(['uid','wa_name'])['up_flow'].sum().unstack().add_prefix('up').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_up_webname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_up_webname['wa_most_up_webname'] = l

In [28]:
# 上传流量最多的a名字
wa_w = wa[wa.wa_type==1]
wa_name = wa_w.groupby(['uid','wa_name'])['up_flow'].sum().unstack().add_prefix('up').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_up_appname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_up_appname['wa_most_up_appname'] = l

In [29]:
# 下载流量最多的w名字
wa_w = wa[wa.wa_type==0]
wa_name = wa_w.groupby(['uid','wa_name'])['down_flow'].sum().unstack().add_prefix('down').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_down_webname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_down_webname['wa_most_down_webname'] = l

In [30]:
# 下载流量最多的a名字
wa_w = wa[wa.wa_type==1]
wa_name = wa_w.groupby(['uid','wa_name'])['down_flow'].sum().unstack().add_prefix('down').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_down_appname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_down_appname['wa_most_down_appname'] = l

In [31]:
# 访问次数最多的w名字
wa_w = wa[wa.wa_type==0]
wa_name = wa_w.groupby(['uid','wa_name'])['visit_cnt'].sum().unstack().add_prefix('cnt').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_cnt_webname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_cnt_webname['wa_most_cnt_webname'] = l

In [32]:
# 访问次数最多的a名字
wa_w = wa[wa.wa_type==1]
wa_name = wa_w.groupby(['uid','wa_name'])['visit_cnt'].sum().unstack().add_prefix('cnt').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_cnt_appname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_cnt_appname['wa_most_cnt_appname'] = l

In [33]:
# 访问时长最多的w名字
wa_w = wa[wa.wa_type==0]
wa_name = wa_w.groupby(['uid','wa_name'])['visit_dura'].sum().unstack().add_prefix('cnt').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_dura_webname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_dura_webname['wa_most_dura_webname'] = l

In [34]:
# 访问时长最多的a名字
wa_w = wa[wa.wa_type==1]
wa_name = wa_w.groupby(['uid','wa_name'])['visit_dura'].sum().unstack().add_prefix('cnt').reset_index().fillna(0)
wa_uid = wa_name.uid;
wa_name = wa_name.drop(['uid'],axis=1)
col_name = np.arange(0,wa_name.shape[1])
col_name = col_name.astype('U')
wa_name.columns =col_name
wa_name_t = wa_name.T
col_name2 = np.arange(0,wa_name_t.shape[1])
col_name2 = col_name2.astype('U')
wa_name_t.columns =col_name2
ss = wa_name_t.idxmax()
wa_most_dura_appname = pd.DataFrame(wa_uid, columns=['uid'])
l = list(ss)
wa_most_dura_appname['wa_most_dura_appname'] = l

In [74]:
# 将各个特征拼在一起
feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,voice_dura,
           voice_opp_len_inout,voice_dura_type,voice_day_count,voice_everyday_out_count,voice_everyday_in_count,
           voice_hour_count,voice_everyday_in_dura,voice_everyday_out_dura,voice_everyday_call_type,voice_everyday_opp_num,
           voice_everyday_opp_head,
           sms_out_opp_num,sms_in_opp_num,sms_opp_head,sms_out_opp_len,sms_in_opp_len,sms_in_out,
           sms_day_count,sms_everyday_out_count,sms_everyday_in_count,sms_hour_out_count,sms_hour_in_count,
           w_a_name_cnt,w_a_visit_cnt,w_a_visit_dura,w_a_upflow,w_a_downflow,
           wa_day_count,wa_everyday_web_up_flow,wa_everyday_web_visit_cnt,wa_everyday_app_visit_cnt,
           wa_everyday_app_up_flow,wa_everyday_web_down_flow,wa_everyday_app_down_flow,
           wa_everyday_web_visit_dura,wa_everyday_app_visit_dura,wa_most_up_webname,wa_most_down_webname,
           wa_most_cnt_webname,wa_most_dura_webname,wa_most_up_appname,wa_most_down_appname,wa_most_cnt_appname,
           wa_most_dura_appname]

In [75]:
# 将train特征和test特征分离
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')

D:\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:558: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
D:\Anaconda\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [76]:
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')

D:\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:558: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
D:\Anaconda\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [77]:
# 填补Nan
train_feature = train_feature.drop(['uid_x'],axis=1)
train_feature = train_feature.fillna(0)
test_feature = test_feature.drop(['uid_x'],axis=1)
test_feature = test_feature.fillna(0)

In [78]:
# 保存特征值
train_feature.to_csv('../data/train_featureV1.csv',index=None)
test_feature.to_csv('../data/test_featureV1.csv',index=None)

In [79]:
test_feature.head()

,uid,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,voice_opp_len_9,...,wa_everyday_app_visit_dura44,wa_everyday_app_visit_dura45,wa_most_up_webname,wa_most_down_webname,wa_most_cnt_webname,wa_most_dura_webname,wa_most_up_appname,wa_most_down_appname,wa_most_cnt_appname,wa_most_dura_appname
0,u7000,4.0,4.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2217182.0,977763.0,2814,12710,1097,2813,752,752,752,752
1,u7001,2.0,3.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,...,45430.0,149707.0,2813,2813,1097,2813,752,752,752,752
2,u7002,16.0,65.0,9.0,0.0,0.0,0.0,0.0,1.0,0.0,...,8933.0,7108.0,9420,2813,11993,2813,752,396,752,752
3,u7003,22.0,90.0,16.0,0.0,1.0,0.0,3.0,0.0,0.0,...,1675.0,59461.0,9420,2813,12468,12468,752,752,752,752
4,u7004,7.0,28.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2744,2744,2743,2744,0,0,0,0
